In [1]:
sc

<SparkContext master=spark://pc-171.calit2.optiputer.net:7077 appName=pyspark-shell>

In [5]:
import os
import sys

import numpy as np

from pyspark.sql import functions as F
from pyspark.sql.functions import col
from pyspark.sql import types as T
from pyspark.ml.linalg import DenseVector, VectorUDT
from pyspark.ml.feature import StringIndexer, StandardScaler

from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K

from elephas.ml_model import ElephasEstimator
from elephas import optimizers as elephas_optimizers
from elephas.ml.adapter import from_data_frame, to_data_frame


/home/mcollins/spark_keras2/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [6]:
#limit to CPU
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = ""


In [7]:
image_df = (sqlContext
             .read
             .parquet("data/idigbio-media-20171112T013207-mercury-images-32x32.parquet")
             .limit(1000)
             .filter(col("imgnpa").isNotNull())
           )
print(image_df.count())
image_df.printSchema()

1000
root
 |-- accessuri: string (nullable = true)
 |-- contaminated: integer (nullable = true)
 |-- imgnpa: array (nullable = true)
 |    |-- element: float (containsNull = true)



In [10]:
renamed_df = image_df.withColumn("features", col("imgnpa")).withColumn("label", col("contaminated"))

# Split in to train/test
splits = renamed_df.randomSplit([0.8, 0.2], 314)
train_df = splits[0]
test_df = splits[1]

print(train_df.count())
print(test_df.count())

814
186


In [11]:
features_train, labels_train = from_data_frame(train_df, True, 2)
features_test, labels_test = from_data_frame(test_df, True, 2)

import numpy as np
from math import sqrt
# Change features to right shape
def raise_dim(l):
    x = np.array(l)
    square_dim = int(sqrt(len(x))) # gave up, hard coded
    r = x[0::3].reshape(-1, 32)
    g = x[1::3].reshape(-1, 32)
    b = x[2::3].reshape(-1, 32)
    return np.array([r, g, b])

features_train = np.array([raise_dim(x) for x in features_train])
features_test = np.array([raise_dim(x) for x in features_test])


In [12]:
# Model
K.set_image_dim_ordering('th')

width, height = 32, 32
if K.image_data_format() == 'channels_first':
    input_shape = (3, width, height)
else:
    input_shape = (width, height, 3)


model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(3, width, height)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(2))
model.add(Activation('sigmoid'))

model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [14]:
# Define elephas optimizer (which tells the model how to aggregate updates on the Spa$
adadelta = elephas_optimizers.Adadelta()

from elephas.utils.rdd_utils import to_labeled_point
from elephas.utils.rdd_utils import to_simple_rdd
lp_rdd = to_simple_rdd(sc, features_train, labels_train)

#print(lp_rdd.take(5))


from elephas.spark_model import SparkModel
from elephas import optimizers as elephas_optimizers

adagrad = elephas_optimizers.Adagrad()
spark_model = SparkModel(sc,model, optimizer=adagrad, frequency='epoch', mode='asynchronous', num_workers=8)
spark_model.train(lp_rdd, nb_epoch=20, batch_size=32, verbose=0, validation_split=0.1)

print(spark_model)


Process Process-1:
Traceback (most recent call last):
  File "/opt/python/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/opt/python/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/mcollins/spark_keras2/lib/python3.6/site-packages/elephas-0.3-py3.6.egg/elephas/spark_model.py", line 171, in start_service
    threaded=True, use_reloader=False)
  File "/home/mcollins/spark_keras2/lib/python3.6/site-packages/Flask-1.0.2-py3.6.egg/flask/app.py", line 938, in run
    cli.show_server_banner(self.env, self.debug, self.name, False)
  File "/home/mcollins/spark_keras2/lib/python3.6/site-packages/Flask-1.0.2-py3.6.egg/flask/cli.py", line 629, in show_server_banner
    click.echo(message)
  File "/home/mcollins/spark_keras2/lib/python3.6/site-packages/click-6.7-py3.6.egg/click/utils.py", line 259, in echo
    file.write(message)
io.UnsupportedOperation: not writable


Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 6 in stage 17.0 failed 4 times, most recent failure: Lost task 6.3 in stage 17.0 (TID 77, 10.1.171.254, executor 0): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/opt/python/lib/python3.6/urllib/request.py", line 1318, in do_open
    encode_chunked=req.has_header('Transfer-encoding'))
  File "/opt/python/lib/python3.6/http/client.py", line 1239, in request
    self._send_request(method, url, body, headers, encode_chunked)
  File "/opt/python/lib/python3.6/http/client.py", line 1285, in _send_request
    self.endheaders(body, encode_chunked=encode_chunked)
  File "/opt/python/lib/python3.6/http/client.py", line 1234, in endheaders
    self._send_output(message_body, encode_chunked=encode_chunked)
  File "/opt/python/lib/python3.6/http/client.py", line 1026, in _send_out

In [28]:
width, height = 32, 32
if K.image_data_format() == 'channels_first':
    input_shape = (3, width, height)
else:
    input_shape = (width, height, 3)


model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [9]:
# Define elephas optimizer (which tells the model how to aggregate updates on the Spark master)
adadelta = elephas_optimizers.Adadelta()

# Initialize SparkML Estimator and set all relevant properties
estimator = ElephasEstimator()
estimator.setFeaturesCol("features")             # These two come directly from pyspark,
estimator.setLabelCol("contaminated")                 # hence the camel case. Sorry :)
estimator.set_keras_model_config(model.to_yaml())       # Provide serialized Keras model
estimator.set_optimizer_config(adadelta.get_config())   # Provide serialized Elephas optimizer
estimator.set_categorical_labels(False)
estimator.set_nb_classes(2)
estimator.set_num_workers(1)  # We just use one worker here. Feel free to adapt it.
estimator.set_nb_epoch(20)
estimator.set_batch_size(128)
estimator.set_verbosity(1)
estimator.set_validation_split(0.15)


ElephasEstimator_405fa93322e95cef5122

In [12]:
fitted_model = estimator.fit(train_df)


Process Process-1:
Traceback (most recent call last):
  File "/opt/python/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/opt/python/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/mcollins/spark_keras2/lib/python3.6/site-packages/elephas-0.3-py3.6.egg/elephas/spark_model.py", line 171, in start_service
    threaded=True, use_reloader=False)
  File "/home/mcollins/spark_keras2/lib/python3.6/site-packages/Flask-1.0.2-py3.6.egg/flask/app.py", line 938, in run
    cli.show_server_banner(self.env, self.debug, self.name, False)
  File "/home/mcollins/spark_keras2/lib/python3.6/site-packages/Flask-1.0.2-py3.6.egg/flask/cli.py", line 629, in show_server_banner
    click.echo(message)
  File "/home/mcollins/spark_keras2/lib/python3.6/site-packages/click-6.7-py3.6.egg/click/utils.py", line 259, in echo
    file.write(message)
io.UnsupportedOperation: not writable


Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 10.0 failed 1 times, most recent failure: Lost task 0.0 in stage 10.0 (TID 10, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/opt/python/lib/python3.6/urllib/request.py", line 1318, in do_open
    encode_chunked=req.has_header('Transfer-encoding'))
  File "/opt/python/lib/python3.6/http/client.py", line 1239, in request
    self._send_request(method, url, body, headers, encode_chunked)
  File "/opt/python/lib/python3.6/http/client.py", line 1285, in _send_request
    self.endheaders(body, encode_chunked=encode_chunked)
  File "/opt/python/lib/python3.6/http/client.py", line 1234, in endheaders
    self._send_output(message_body, encode_chunked=encode_chunked)
  File "/opt/python/lib/python3.6/http/client.py", line 1026, in _send_o